RAVDESS

In [15]:
import pandas as pd
import glob
import os
import librosa
import json
from fast_ml.model_development import train_valid_test_split

def get_emotion_label(audio_filepath):
    filename = os.path.basename(audio_filepath)
    identifiers = filename.split('.')[0].split('-')
    emotion = (int(identifiers[2]))
    if emotion==8:
        return 0
    return emotion

#Create Dataframe
df = pd.DataFrame()
df['audio_file_path'] = []
df['emotion'] = []

EMOTION_M = {
    1:'neutral',
    2:'calm', 
    3:'happy', 
    4:'sad', 
    5:'angry', 
    6:'fear', 
    7:'disgust', 
    0:'surprise'
}

#Concat Data

for i in range(1,25):
    folder_path = f'data/Audio_Speech_Actors_01-24/Actor_{str(i).zfill(2)}'
    for audio_file in glob.glob(os.path.join(folder_path, '*.wav')):
                emotion_label = get_emotion_label(audio_file)  # Implement this function
                data = {
                    'audio_filepath': audio_file,
                    # 'duration': get_audio_duration(audio_file),  # Implement this function
                    'label': emotion_label
                }
                new_row = pd.DataFrame({'audio_file_path':[data['audio_filepath']],'emotion':[EMOTION_M[data['label']]]})
                df = pd.concat([df,new_row],ignore_index=True)
print(df)



                                        audio_file_path   emotion
0     data/Audio_Speech_Actors_01-24/Actor_01/03-01-...   neutral
1     data/Audio_Speech_Actors_01-24/Actor_01/03-01-...      fear
2     data/Audio_Speech_Actors_01-24/Actor_01/03-01-...  surprise
3     data/Audio_Speech_Actors_01-24/Actor_01/03-01-...     angry
4     data/Audio_Speech_Actors_01-24/Actor_01/03-01-...      fear
...                                                 ...       ...
1435  data/Audio_Speech_Actors_01-24/Actor_24/03-01-...  surprise
1436  data/Audio_Speech_Actors_01-24/Actor_24/03-01-...     angry
1437  data/Audio_Speech_Actors_01-24/Actor_24/03-01-...  surprise
1438  data/Audio_Speech_Actors_01-24/Actor_24/03-01-...   neutral
1439  data/Audio_Speech_Actors_01-24/Actor_24/03-01-...      fear

[1440 rows x 2 columns]


# to mono

In [16]:
from pydub import AudioSegment

for i in range(1,25):
    folder_path = f'data/Audio_Speech_Actors_01-24/Actor_{str(i).zfill(2)}'
    for audio_file in glob.glob(os.path.join(folder_path, '*.wav')):
            print(audio_file)
            sound = AudioSegment.from_wav(audio_file)
            sound = sound.set_channels(1)
            sound.export(audio_file, format="wav")

data/Audio_Speech_Actors_01-24/Actor_01/03-01-01-01-02-02-01.wav
data/Audio_Speech_Actors_01-24/Actor_01/03-01-06-01-02-02-01.wav
data/Audio_Speech_Actors_01-24/Actor_01/03-01-08-01-01-02-01.wav
data/Audio_Speech_Actors_01-24/Actor_01/03-01-05-02-01-02-01.wav
data/Audio_Speech_Actors_01-24/Actor_01/03-01-06-02-01-01-01.wav
data/Audio_Speech_Actors_01-24/Actor_01/03-01-04-02-01-01-01.wav
data/Audio_Speech_Actors_01-24/Actor_01/03-01-01-01-01-02-01.wav
data/Audio_Speech_Actors_01-24/Actor_01/03-01-04-01-02-01-01.wav
data/Audio_Speech_Actors_01-24/Actor_01/03-01-02-01-01-02-01.wav
data/Audio_Speech_Actors_01-24/Actor_01/03-01-02-01-01-01-01.wav
data/Audio_Speech_Actors_01-24/Actor_01/03-01-05-02-02-01-01.wav
data/Audio_Speech_Actors_01-24/Actor_01/03-01-05-02-01-01-01.wav
data/Audio_Speech_Actors_01-24/Actor_01/03-01-04-02-02-02-01.wav
data/Audio_Speech_Actors_01-24/Actor_01/03-01-08-02-01-02-01.wav
data/Audio_Speech_Actors_01-24/Actor_01/03-01-02-01-02-01-01.wav
data/Audio_Speech_Actors_

# CREMA-D

In [17]:
import glob
import os
import pandas as pd


EMOTION_M = {
    1:'neutral',
    2:'calm', 
    3:'happy', 
    4:'sad', 
    5:'angry', 
    6:'fear', 
    7:'disgust', 
    0:'surprise'
}

def get_emotion_label_CREMA(audio_filepath):
    MAPP = {
        'SAD' : 'sad',
        'ANG' : 'angry',
        'DIS' : 'disgust',
        'FEA' : 'fear',
        'HAP' : 'happy',
        'NEU' : 'neutral'
    }
    filename = os.path.basename(audio_filepath)
    identifiers = filename.split('_')[2]
    if not identifiers in MAPP:
        return [0,False]
    emotion = MAPP[identifiers]
    return [emotion,True]


folder_path_CREMA = 'data/CREMAD'
for audio_file in glob.glob(os.path.join(folder_path_CREMA, '*.wav')):
                emotion_label,unk = get_emotion_label_CREMA(audio_file)  # Implement this function
                if not unk:
                       continue
                data = {
                    'audio_filepath': audio_file,
                    # 'duration': get_audio_duration(audio_file),  # Implement this function
                    'label': emotion_label
                }
                new_row = pd.DataFrame({'audio_file_path':[data['audio_filepath']],'emotion':[data['label']]})
                df = pd.concat([df,new_row],ignore_index=True)

# proceed

In [18]:
#Split data
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df,
                                                                            target = 'emotion',
                                                                            train_size=0.6, 
                                                                            valid_size=0.2, 
                                                                            test_size=0.2)

print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)


for data in [X_train, y_train, X_valid, y_valid, X_test, y_test]:
    data.reset_index(drop=True, inplace=True)

#Write in Manifest file
def Write_Manifest(output_file,dataset_x,dataset_y):
    with open(output_file, "w") as outfile:
        zipped = zip(dataset_x["audio_file_path"].tolist(),dataset_y.tolist())
        for (path,emo) in zipped:
            data = {"audio_filepath": path, "label": emo}
            outfile.write(json.dumps(data) + '\n')
        outfile.close()

Write_Manifest("train_manifest.json", X_train, y_train)
Write_Manifest("validation_manifest.json", X_valid, y_valid)
Write_Manifest("test_manifest.json", X_test, y_test)

(5329, 1) (5329,)
(1776, 1) (1776,)
(1777, 1) (1777,)


# adding duration


In [19]:
import librosa
import json
with open('train_manifest.json') as fin:
    a = []
    for line in fin:
        d = json.loads(line)
        fp = d['audio_filepath']
        

        nd = {}
        nd['audio_filepath'] = fp
        nd['duration'] = librosa.core.get_duration(filename=fp)
        nd['label'] = d['label']

        
        s = json.dumps(nd)
        a.append(s)

    with open('train_manifest.json', "w") as outfile:
        for i in a:
            outfile.write(i + '\n')
        outfile.close()

In [20]:
with open('test_manifest.json') as fin:
    a = []
    for line in fin:
        d = json.loads(line)
        fp = d['audio_filepath']
        

        nd = {}
        nd['audio_filepath'] = fp
        nd['duration'] = librosa.core.get_duration(filename=fp)
        nd['label'] = d['label']

        
        s = json.dumps(nd)
        a.append(s)

    with open('test_manifest.json', "w") as outfile:
        for i in a:
            outfile.write(i + '\n')
        outfile.close()

In [21]:
with open('validation_manifest.json') as fin:
    a = []
    for line in fin:
        d = json.loads(line)
        fp = d['audio_filepath']
        

        nd = {}
        nd['audio_filepath'] = fp
        nd['duration'] = librosa.core.get_duration(filename=fp)
        nd['label'] = d['label']

        
        s = json.dumps(nd)
        a.append(s)

    with open('validation_manifest.json', "w") as outfile:
        for i in a:
            outfile.write(i + '\n')
        outfile.close()